In [1]:
import gym 
import retro
import numpy as np
import  gym
from gym import Env


from torch.utils.tensorboard import SummaryWriter
import retro
import cv2
from gym.spaces import Box,MultiBinary
import numpy as np
import optuna
import tqdm as notebook_tqdm
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder ,VecFrameStack
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class KungFuKid(Env):
    def __init__(self) -> None:
        super().__init__()
        self.observation_space=Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space=MultiBinary(9)
        self.game = retro.make(game='FantasyZoneTheMaze-Sms',use_restricted_actions=retro.Actions.FILTERED)

    def preprocess_observation(self,obs):
        gray = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
        resize = cv2.resize(gray, (84, 84),interpolation=cv2.INTER_AREA)
        chennels = np.expand_dims(resize, axis=2)
        return chennels
   

    def step(self, action):
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess_observation(obs)
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs

        score_reward = (info['score'] - self.score) / 10

        # Lives punishment
        lives_punishment = 0
        if info['lives'] < self.lives:
            lives_punishment = 10

        reward = score_reward - lives_punishment

        self.score = info['score']
        self.lives = info['lives']

        return frame_delta, reward, done, info


    def reset(self):
        obs = self.game.reset()
        obs = self.preprocess_observation(obs)
        self.previous_frame = obs
        self.score = 0
        self.lives = 5
        return obs

    def render (self,*args,**kwargs):
        self.game.render()

    def close(self):
        self.game.close()


In [3]:
import time

best_model_10000.zip

In [4]:
model_dir='D:/torch_rl/AI_FOR_games/maze/models/FantasyZoneTheMaze-Sms/best_model_15580000.zip'
# model_dir='D:/torch_rl/AI_FOR_games/maze/models/FantasyZoneTheMaze-Sms/best_model_10000.zip'

In [5]:
model = PPO.load(model_dir)

In [6]:
env = KungFuKid()
env = Monitor(env)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [7]:
obs = env.reset()
done= False
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    time.sleep(0.001)
    

In [8]:
rewards

array([-10.], dtype=float32)